### Load labware

In [32]:
import opentrons.execute
from opentrons import protocol_api

metadata = {'apiLevel': '2.13'}

In [35]:
from opentrons import simulate
#protocol = simulate.get_protocol_api(metadata['apiLevel']) # uncomment to run in simulation

protocol = opentrons.execute.get_protocol_api(metadata['apiLevel']) # uncomment to run on the actual hardware

In [36]:
mag_deck = protocol.load_module('magneticModuleV2', 1)
mag_plate = mag_deck.load_labware(
    'biorad_96_wellplate_200ul_pcr')
block = protocol.load_labware('opentrons_24_aluminumblock_nest_1.5ml_snapcap', 2)
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 5)
pipette = protocol.load_instrument('p300_single_gen2', 'right',
                                     tip_racks=[tiprack])
pipette.starting_tip = tiprack['A1'] # TODO: change to the starting position of your tiprack, tips are taken column-wise

tiprack_20 = tiprack = protocol.load_labware('opentrons_96_tiprack_20ul', 4)
p20 = protocol.load_instrument('p20_single_gen2', 'left', tip_racks=[tiprack_20])
p20.starting_tip = tiprack_20['E2']

### Define inputs and volumes

In [37]:
# Define reagents and liquid waste
sample = block['B1']
beads = block['A1']
beads_well = mag_plate['H6']
output = block['D6']

wash_buffer = block['A2']
elution_buffer = block['A3']
binding_buffer = block['A4']
liquid_waste = block['A5']


# Define bead and mix volume
sample_volume = 10
bead_ratio = 1.8
binding_buf_volume = 90
elution_buf_volume = 20
washing_buf_volume = 100
beads_volume = sample_volume*bead_ratio

total_vol = beads_volume + binding_buf_volume + sample_volume

# define waiting times
settling_time = 1 # minute
incubation_time = 5 # minutes
drying_time = 40 # minutes

### Helper functions

In [22]:
def remove_supernatant(volume, source, dest, asp_rate=0.2):
    """ Remove supernatant carefully, with lower aspiration rate to not disturb the pellet"""
    pipette.pick_up_tip()
    pipette.aspirate(volume+10, source, rate=asp_rate) # add extra 10ul to make sure we aspirate everything
    pipette.dispense(volume+10, dest)
    pipette.blow_out()
    pipette.drop_tip()

### Prepare Beads

In [23]:
pipette.home()
pipette.pick_up_tip()
pipette.mix(10, 150, beads)
pipette.drop_tip()
p20.transfer(beads_volume, beads, beads_well)

<InstrumentContext: p20_single_v2.1 in LEFT>

In [25]:

mag_deck.engage()
# wait for beads to separate
protocol.delay(minutes=settling_time) 

# wash supernatant
p20.transfer(beads_volume, beads_well, liquid_waste)
mag_deck.disengage()

# wash twice with elution buffer
for _ in range(2):
    pipette.transfer(100, elution_buffer, beads_well, mix_after=(10, 50))
    mag_deck.engage()
    protocol.delay(minutes=settling_time)
    remove_supernatant(100, beads_well, liquid_waste)
    mag_deck.disengage()

### Purify Plasmid

In [26]:
# Mix sample with binding buffer
pipette.pick_up_tip()
pipette.mix(20, 150, sample)
pipette.drop_tip()
p20.transfer(sample_volume, sample, beads_well) 
pipette.transfer(binding_buf_volume, binding_buffer, beads_well, mix_after=(10, 50))

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [27]:
# Incubate beads and sample at room temperature
protocol.delay(minutes=incubation_time) # Note: original protocol incubates while gently mixing, so this step could be a problem


In [28]:
# Engagae MagDeck and incubate
mag_deck.engage()
protocol.delay(minutes=settling_time)

In [29]:
# Remove supernatant from magnetic beads
remove_supernatant(total_vol, beads_well, liquid_waste)


In [30]:
# Wash beads twice with wash buffer
air_vol = pipette.max_volume * 0.1
wash_buf_volume = 100
mix_vol = wash_buf_volume / 2
for _ in range(2):
    mag_deck.disengage()
    pipette.transfer(wash_buf_volume, wash_buffer, beads_well, air_gap=air_vol, mix_after=(10, mix_vol))
    mag_deck.engage()
    protocol.delay(minutes=settling_time)
    pipette.transfer(wash_buf_volume, beads_well, liquid_waste, air_gap=air_vol) # TODO: do I need to be careful with aspiration rate here?


In [ ]:
# Disengage MagDeck
mag_deck.disengage()

# Dry at RT
protocol.delay(minutes=drying_time)

## Elute and collect the plasmid

In [38]:
mix_vol = elution_buf_volume/2
pipette.transfer(elution_buf_volume, elution_buffer, beads_well, mix_after=(20, mix_vol))


<InstrumentContext: p300_single_v2.1 in RIGHT>

In [39]:

# Incubate at RT
protocol.delay(minutes=incubation_time) # TODO: we need some mixing here

In [47]:
# Engage MagDeck and remain engaged for DNA elution
mag_deck.engage()
protocol.delay(minutes=settling_time)

# Transfer clean PCR product to a new well
remove_supernatant(elution_buf_volume, beads_well, output)


In [13]:
def test_wb_1():
    pipette.home()
    #pipette.pick_up_tip()
    #pipette.mix(10, 150, beads)
    #pipette.drop_tip()
    p20.transfer(beads_volume, beads, beads_well)

def test_wb_2():
    mag_deck.engage()
    # wait for beads to separate
    protocol.delay(minutes=settling_time) 
    # wash supernatant
    p20.transfer(beads_volume, beads_well, liquid_waste)
    mag_deck.disengage()
    
def test_wb_3():
    # Wash beads twice with wash buffer
    air_vol = pipette.max_volume * 0.1
    wash_buf_volume = 100
    mix_vol = wash_buf_volume / 2
    for _ in range(1):
        mag_deck.disengage()
        pipette.transfer(wash_buf_volume, wash_buffer, beads_well, air_gap=air_vol, mix_after=(10, mix_vol))
        mag_deck.engage()
        protocol.delay(minutes=settling_time)
        pipette.transfer(wash_buf_volume, beads_well, liquid_waste, air_gap=air_vol)
    mag_deck.disengage()
    
def test_elution_1():
    mix_vol = elution_buf_volume/2
    pipette.transfer(elution_buf_volume, elution_buffer, beads_well, mix_after=(20, mix_vol))
    mag_deck.engage()
    protocol.delay(minutes=settling_time)

def test_elution_2():
    remove_supernatant(elution_buf_volume, beads_well, output)